### Diseño de Agentes Inteligentes
### Tarea 2. Agentes Solucionadores de Problemas (PSA) y búsqueda

*Integrantes del Equipo 4* 
1. Ricardo Kaleb Flores Alfonso     A01198716
2. Sebastián Miramontes Soto        A01285296
3. Raúl Correa Ocañas               A01722401


#### Cleanup Puzzle

Cleanup Puzzle es un juego de mesa que requiere que limpies el tablero. Para ello, deberás hacer clic en las fichas de un tablero tipo cuadrícula. Cada ficha puede estar vacía o contener una bola. El mosaico en el que haga clic no se verá afectado, pero sus vecinos directos (horizontal y verticalmente) se invertirán (si el mosaico contiene una bola, se vaciará y viceversa). El objetivo es determinar una secuencia de clics que elimine todas las bolas del tablero.

El juego, que puedes encontrar y jugar en el enlace https://www.mathsisfun.com/games/cleanup-puzzle.html , consiste en un tablero de 11x11 que se juega en una secuencia de etapas de complejidad incrementada. La complejidad de un escenario está relacionada con la cantidad de bolas en el tablero y sus ubicaciones. En el juego en línea, hay un límite de clics definido para cada etapa y el juego termina cuando no puede encontrar una secuencia de clics lo suficientemente pequeña para limpiar el tablero.
Ejemplos de diferentes etapas del juego en línea:

![alt text](ejemplo1.png)

![alt text](ejemplo2.png)


In [1]:
# Simpleai.search es la librería desde la que importaremos los algoritmos de búsqueda
from simpleai.search import SearchProblem, astar, breadth_first, depth_first, uniform_cost
# BaseViewer nos ayuda a resumir los resultados de la búsqueda
from simpleai.search.viewers import BaseViewer, WebViewer

PSA de Busqueda Informada utilizando el algoritmo A*

In [2]:
class Cleanup(SearchProblem):
    # Definimos el problema de búsqueda. Cleanup hereda de SearchProblem y toma como parametros
    # el estado inicial y el estado meta
    def __init__(self, initial, goal):
        super().__init__(initial_state = initial)
        self.goal_state = goal

    def actions(self, state):
        valid_actions = []
        rows = len(state)
        cols = len(state[0])

        for i in range(rows):
            for j in range(cols):
                # Check if the cell has neighbors (horizontally or vertically)
                if (i > 0) or (i < rows - 1) or (j > 0) or (j < cols - 1):
                    valid_actions.append((i, j))
        
        return valid_actions

    
    def result(self, state, action):
        i, j = action
        child_state = [list(row) for row in state]
        
        if i - 1 >= 0:
            child_state[i - 1][j] = not child_state[i - 1][j]
        if j - 1 >= 0:
            child_state[i][j - 1] = not child_state[i][j - 1]
        if j + 1 < len(state[0]):
            child_state[i][j + 1] = not child_state[i][j + 1]
        if i + 1 < len(state):
            child_state[i + 1][j] = not child_state[i + 1][j]

        return tuple([tuple(row) for row in child_state])
    
    def is_goal(self, state):
        return state == self.goal_state
    
    def cost(self, state, action, state2):
        return 1
    
    def heuristic(self, state):
        rows = len(state)
        cols = len(state[0])
        ball_positions = []

        for i in range(rows):
            for j in range(cols):
                if state[i][j]:
                    ball_positions.append((i, j))

        heuristic_value = 0
        for pos1 in ball_positions:
            min_distance = float("inf")
            for pos2 in ball_positions:
                if pos1 != pos2:
                    distance = abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
                    min_distance = min(min_distance, distance)
            heuristic_value += min_distance

        return heuristic_value


In [3]:
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i, action)
                print('¡Meta lograda con costo = ', result.cost,'!')
            else:
                print(i, action)

            print('  ', state)
    else:
        print('Mala configuración del problema')

In [14]:
rows, cols = 11,11
def list_board(rows, cols):
        return [[False for i in range(cols)] for j in range(rows)]

def tuple_board(board):
        return tuple([tuple(row) for row in board])

init = list_board(rows, cols)

# Dificultad 1
init[3][0] = True
init[4][1] = True
init[5][0] = True

init[10][8] = True
init[9][9] = True
init[10][10] = True


# Dificultad 2
# init[2][4] = True
# init[3][3] = True
# init[3][5] = True
# init[4][4] = True

# init[7][0] = True
# init[6][1] = True
# init[8][1] = True

# init[10][4] = True
# init[9][5] = True

# init[7][4] = True
# init[6][3] = True
# init[8][3] = True

goal = list_board(rows, cols)

init = tuple_board(init)
goal = tuple_board(goal)

init_int = [[int(cell) for cell in row] for row in init]
init_int

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]]

In [15]:
my_viewer = BaseViewer()       # Solo estadísticas
# my_viewer = WebViewer()      # Estadísticas y visualización del árbol de búsqueda

result = astar(Cleanup(init,goal), graph_search=True, viewer=my_viewer)
# result = breadth_first(Cleanup(init,goal), graph_search=True, viewer=my_viewer)
# result = depth_first(Cleanup(init,goal), graph_search=True, viewer=my_viewer)
# result = uniform_cost(Cleanup(init,goal), graph_search=True, viewer=my_viewer)

if my_viewer != None:
    print('Stats:')
    print(my_viewer.stats)

print()
print('>> Búsqueda A* <<')
display(result)


Stats:
{'max_fringe_size': 240, 'visited_nodes': 3, 'iterations': 3}

>> Búsqueda A* <<
Configuración inicial
   ((False, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, False, False), (True, False, False, False, False, False, False, False, False, False, False), (False, True, False, False, False, False, False, False, False, False, False), (True, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, False, False), (False, False, False, False, False, False, False, False, False, True, False), (False, False, False, False, False, False, False, False, True, False, True))
1 (4, 0)
   ((False, Fa